# Import library

In [21]:
import numpy as np
import pandas as pd
import psycopg2 as pg
from datetime import datetime, timedelta
import calendar
import time

# DB connection

In [2]:
conn = pg.connect("dbname=demo user=postgres password=kumden090604")
cur = conn.cursor()

# Get Data

In [3]:
# PYTHON
import requests

def get_data_vn30f1m():
    api = "http://192.168.30.109:8006/api/v1/live?symbol=vn30f1m&start_date=2026-01-22&interval=1m"
    for i in range(5):
        try:
            response = requests.get(api)
            # print(response.json())
            if response.status_code == 200:
                data_json = response.json()
                if data_json["code"] == 200:
                    # Chuyển đổi dữ liệu trả về sang Dataframe
                    flattened_data = [item for batch in data_json["data"] for item in batch]
                    df = pd.DataFrame(flattened_data)
                    return df
                    # print(df)
                    # break
                else:
                    print(data_json["message"])
            else:
                print(response.text)
        except Exception as e:
            print(e)

df_vn30f1m = get_data_vn30f1m()
df_vn30f1m

,datetime,open,high,low,close,volume,value
0,2026-01-22T09:00:00,2089.4,2091.9,2088.0,2088.8,5098.0,10648702.4
1,2026-01-22T09:01:00,2088.8,2093.9,2088.8,2089.8,2624.0,5483635.2
2,2026-01-22T09:02:00,2089.8,2091.4,2089.4,2090.7,2001.0,4183490.7
3,2026-01-22T09:03:00,2091.0,2092.7,2090.5,2091.3,1307.0,2733329.1
4,2026-01-22T09:04:00,2091.3,2092.8,2091.1,2092.8,939.0,1965139.2
...,...,...,...,...,...,...,...
1917,2026-02-03T09:33:00,2022.2,2024.0,2022.0,2023.7,1247.0,2523553.9
1918,2026-02-03T09:34:00,2023.7,2023.9,2023.1,2023.6,563.0,1139286.8
1919,2026-02-03T09:35:00,2023.6,2024.3,2022.3,2022.4,1151.0,2327782.4
1920,2026-02-03T09:36:00,2022.5,2022.8,2021.5,2021.7,750.0,1516275.0


# Make a copy of the original data

In [151]:
dummy = df_vn30f1m.copy()


# Alter data for testing

In [152]:
dummy.loc[3,'datetime'] = '2026-01-23T19:45:00'
dummy.loc[4,'high'] = np.nan
dummy = dummy.drop(2)
new = {"datetime":"2026-12-25T13:20:00", "open": 67,"high": 67,"low": 67,"close":67,"volume": 67,"value":67}
dummy.loc[len(dummy)] = new
dummy

,datetime,open,high,low,close,volume,value
0,2026-01-22T09:00:00,2089.4,2091.9,2088.0,2088.8,5098.0,10648702.4
1,2026-01-22T09:01:00,2088.8,2093.9,2088.8,2089.8,2624.0,5483635.2
3,2026-01-23T19:45:00,2091.0,2092.7,2090.5,2091.3,1307.0,2733329.1
4,2026-01-22T09:04:00,2091.3,NaN,2091.1,2092.8,939.0,1965139.2
5,2026-01-22T09:05:00,2092.8,2093.6,2091.6,2092.6,1638.0,3427678.8
...,...,...,...,...,...,...,...
1917,2026-02-03T09:33:00,2022.2,2024.0,2022.0,2023.7,1247.0,2523553.9
1918,2026-02-03T09:34:00,2023.7,2023.9,2023.1,2023.6,563.0,1139286.8
1919,2026-02-03T09:35:00,2023.6,2024.3,2022.3,2022.4,1151.0,2327782.4
1920,2026-02-03T09:36:00,2022.5,2022.8,2021.5,2021.7,750.0,1516275.0


# Testing

In [231]:
class DMBase:

        def check_data(
                df: pd.DataFrame,
                asset_class_id: str,
                interval: int
        )-> pd.DataFrame:
                query = """
                SELECT session_time_start, session_time_end
                FROM asset ass
                JOIN asset_time ast ON ass.asset_name = ast.asset_name
                WHERE asset_class_id = %s
                """

                cur.execute(query, (asset_class_id,))
                ds = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])
                df['datetime'] = pd.to_datetime(df['datetime'])
                df['datetime'].dt.strftime("%Y-%m-%d %H:%M:%S")

                sessions = ds[['session_time_start', 'session_time_end']].apply(pd.to_datetime)
                query = """
                SELECT holiday_date_start,holiday_date_end
                FROM holiday_calendar
                WHERE asset_class_id = %s
                """
                cur.execute(query, ("vn30f1m",))
                dz = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])

                dz['holiday_date_start'] = pd.to_datetime(dz['holiday_date_start'])
                holiday_dates = set(dz['holiday_date_start'].dt.date)
                # Remove the holiday
                df = df[
                ~df['datetime'].dt.date.isin(holiday_dates)
                ]
                start_date = df['datetime'].dt.normalize().min()
                end_date   = df['datetime'].dt.normalize().max()
                # Get all the valid days
                all_days = [
                day for day in pd.date_range(start=start_date, end=end_date, freq='D')
                if day.date() not in holiday_dates
                ]

                full_index = []
                # Get all sessions of each day
                for day in all_days:
                        # Session 1
                        s1_start = day + pd.Timedelta(
                                hours=sessions.loc[0, 'session_time_start'].hour,
                                minutes=sessions.loc[0, 'session_time_start'].minute
                        )
                        s1_end = day + pd.Timedelta(
                                hours=sessions.loc[0, 'session_time_end'].hour,
                                minutes=sessions.loc[0, 'session_time_end'].minute
                        )

                        # Session 2
                        s2_start = day + pd.Timedelta(
                                hours=sessions.loc[1, 'session_time_start'].hour,
                                minutes=sessions.loc[1, 'session_time_start'].minute
                        )
                        s2_end = day + pd.Timedelta(
                                hours=sessions.loc[1, 'session_time_end'].hour,
                                minutes=sessions.loc[1, 'session_time_end'].minute
                        )
                        
                        full_index.append(pd.date_range(s1_start, s1_end, freq=f'{interval}min'))
                        full_index.append(pd.date_range(s2_start, s2_end, freq=f'{interval}min'))
                
                # Fill in missing data and remove dup
                full_index = pd.concat(
                [pd.Series(idx) for idx in full_index],
                ignore_index=True
                ).sort_values().drop_duplicates()

                full_index = pd.DatetimeIndex(full_index)

                df = dummy.set_index('datetime')
                df = df.reindex(full_index)
                df = df.ffill()

                return df
        # Check if today is valid
        @staticmethod
        def is_trading_day(
                asset_class_id: str
        )-> bool:
                query_days = """
                SELECT distinct days_of_month
                FROM asset ass
                JOIN asset_time ast ON ass.asset_name = ast.asset_name
                WHERE asset_class_id = %s
                """
                query_holiday = """
                SELECT distinct holiday_date_start
                FROM holiday_calendar
                WHERE asset_class_id = %s
                """
                cur.execute(query_days, (asset_class_id,))
                dc = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])
                
                cur.execute(query_holiday, (asset_class_id,))
                dz = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])
                
                x = datetime.now().weekday()
                y = datetime.now().date()
                x1 = dc.loc[0,"days_of_month"].split("-")[0]
                y1 = dc.loc[0,"days_of_month"].split("-")[1]
                x_as_int = time.strptime(x1, "%A").tm_wday
                y_as_int = time.strptime(y1, "%A").tm_wday
                if (x >= x_as_int and x <= y_as_int):
                        return not bool((dz['holiday_date_start'] == y).any())
                                
                
        # Check if current time is valid
        @staticmethod
        def is_trading_time(
                asset_class_id: str
        )-> bool:
                query = """
                SELECT session_time_start, session_time_end
                FROM asset ass
                JOIN asset_time ast ON ass.asset_name = ast.asset_name
                WHERE asset_class_id = %s
                """

                cur.execute(query, (asset_class_id,))
                ds = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])
                sessions = ds[['session_time_start', 'session_time_end']].apply(pd.to_datetime)
                now = datetime.now().time()
                res = any(
                start.time() <= now <= end.time()
                for start, end in zip(sessions.iloc[:, 0], sessions.iloc[:, 1])
                )
                return res
 

In [154]:
DMBase.check_data(dummy,"vn30f1m",1)

,open,high,low,close,volume,value
2026-01-22 09:00:00,2089.4,2091.9,2088.0,2088.8,5098.0,10648702.4
2026-01-22 09:01:00,2088.8,2093.9,2088.8,2089.8,2624.0,5483635.2
2026-01-22 09:02:00,2088.8,2093.9,2088.8,2089.8,2624.0,5483635.2
2026-01-22 09:03:00,2088.8,2093.9,2088.8,2089.8,2624.0,5483635.2
2026-01-22 09:04:00,2091.3,2093.9,2091.1,2092.8,939.0,1965139.2
...,...,...,...,...,...,...
2026-02-03 14:41:00,2022.5,2022.8,2021.5,2021.7,750.0,1516275.0
2026-02-03 14:42:00,2022.5,2022.8,2021.5,2021.7,750.0,1516275.0
2026-02-03 14:43:00,2022.5,2022.8,2021.5,2021.7,750.0,1516275.0
2026-02-03 14:44:00,2022.5,2022.8,2021.5,2021.7,750.0,1516275.0


In [234]:
DMBase.is_trading_day("vn30f1m")

True

In [155]:
DMBase.is_trading_time("vn30f1m")

True